# 전처리 종합실습 평가 ③

#### 목표
1. 간략히 가설수립을 합니다.
2. 데이터를 탐색합니다.(역시 간략히)
3. 가설로 도출된 요인들을 추가변수로 생성합니다.
4. 모델링을 위한 전처리를 수행합니다.
5. 준비된 모델링 코드로 성능을 확인합니다.

## 1. 비즈니스 이해

문제 정의 : 
* 우리는 고객이 이탈할 것인지 사전에 예측하는 모델을 만들고자 합니다.
* 이탈할 고객을 미리 예측할 수 있다면, 프로모션을 통해 이탈을 예방할 수 있습니다.
* 그렇다면 고객의 이탈에 영향을 주는 요인은 무엇일까요?


다음과 같이 구체적으로 문제를 정의하고자 한다.
* 2016년12월말 기준으로
* 이후 3개월 방문(구매) 안한 사람을 이탈로 보자.
* 관심있는 고객은 
    - 2015~2016년 신규 가입 고객 이면서,
    - 2016년 하반기에 한번 이상 방문한 고객


### 문제1 :  
아래 셀에 이탈여부와 관련한 요인을 최소한 10가지 이상을 도출해 봅시다.  
이 질문에 대한 답은 2.데이터이해 단계와 3.데이터준비단계를 수행하면서 다시 여기로 돌아와서 수정하거나 추가해야 할 수도 있습니다.


예시1) 2016년에 4번이상 방문하고 10000만원 이상구매한 고객은 이탈하지 않을 것이다.  
예시2) 남성고객이면서, 30대 고객은 고객은 이탈할 것이다.   
    
    
    1)
    2)
    3)
    4)
    5)
    6)
    7)
    8)
    9)
    10)


## 2.데이터 이해

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers.csv'
                 , sep=',', skipinitialspace=True, encoding='CP949')  
customers.head()

,CustomerID,RegisterDate,Address,Gender,BirthYear,Addr1,Addr2
0,c328222,2014-09-25,강원 원주시 늘품로,F,1960,강원도,원주시
1,c281448,2013-06-18,강원 원주시 치악로,F,1974,강원도,원주시
2,c038336,2003-10-10,강원 춘천시 서부대성로,F,1968,강원도,춘천시
3,c084237,2007-03-09,강원도 강릉시 연곡면 황어대길,F,1982,강원도,강릉시
4,c162600,2010-06-14,강원도 속초시 농공단지길,F,1978,강원도,속초시


In [7]:
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv'
                 , sep=',', skipinitialspace=True, encoding='CP949')  
products.head()

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [8]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv'
                 , sep=',', skipinitialspace=True, encoding='CP949')  
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [9]:
sales['OrderDate'] = pd.to_datetime(sales['OrderDate']) 
customers['RegisterDate'] = pd.to_datetime(customers['RegisterDate']) 

**데이터 탐색은 3.데이터 준비 단계 후에 진행해 봅시다.**

###  문제 2. 데이터 살펴보기

2-1) 연도별, 월별 매출액은?

In [10]:
sales['Year'] = sales.OrderDate.dt.year
sales['Month'] = sales.OrderDate.dt.month

In [11]:
sales.groupby(by=['Year', 'Month'], as_index = False)['Amt'].sum()

,Year,Month,Amt
0,2016,1,14209990
1,2016,2,14367451
2,2016,3,17111120
3,2016,4,15129917
4,2016,5,13976941
5,2016,6,13113610
6,2016,7,10717743
7,2016,8,10176702
8,2016,9,10496653
9,2016,10,10003648


2-2) 총 가입고객수는?

In [12]:
customers.CustomerID.count()

2243

2-3) 2015~2016에 가입한 고객은 총 몇명인가?

In [13]:
customers.loc[customers.RegisterDate.between('2015-01-01', '2016-12-31'),['CustomerID']].count()

CustomerID    485
dtype: int64

In [14]:
customers.loc[customers.RegisterDate.between('2015-01-01', '2016-12-31'),]['CustomerID'].count()

485

2-4) 2016년 하반기에 구매한 적이 있는 고객의 수는?

In [15]:
sales.loc[sales['OrderDate'].between('2016-07-01', '2016-12-31'),].CustomerID.unique().size

1711

2-5) 위에서 조회한 내용이 아닌 가설과 관련된 요인에 대해서 두가지를 조회하시오. 
가설의 내용을 그대로 조회하면 좋지만, 가설에 포함된 요인을 다양한 조건으로 조회해 보면 됩니다.

In [ ]:
# #번가설 :      




In [ ]:
# #번가설 :      




In [ ]:
# #번가설 :      




## 3. 데이터 준비(Preprocessing)

### 문제 3. Labeling

우리는 고객이 이탈할 것인지 사전에 예측하는 모델을 만들고자 한다.  

* 2016년12월말 기준
* 이후 3개월 방문 안한 사람을 이탈로 보자.
* 관심있는 고객은 
    - 2015~2016년 신규 가입 고객 이면서,
    - 2016년 하반기에 한번 이상 방문한 고객

3-1) 고객ID 조회 : 
    - 2016년 하반기에 한번 이상 방문한 고객 중 2015~2016년 신규 가입 고객을 조회하시오.

In [16]:
sales.loc[sales['OrderDate'] >= '2017-01-01',]

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
55976,5,2,2017-01-02,p1133371,7,7000,c271068,2017,1
55977,5,6,2017-01-02,p1012751,1,1850,c271068,2017,1
55978,5,10,2017-01-02,p1144661,1,1950,c271068,2017,1
55979,5,11,2017-01-02,p1207281,1,3300,c271068,2017,1
55980,5,12,2017-01-02,p1299491,1,1950,c271068,2017,1
...,...,...,...,...,...,...,...,...,...
70007,53,4,2017-03-31,p1072601,1,4600,c337999,2017,3
70008,53,6,2017-03-31,p1178011,1,8800,c337999,2017,3
70009,55,6,2017-03-31,p1054261,1,2091,c088320,2017,3
70010,59,4,2017-03-31,p1175481,1,1300,c238056,2017,3


In [17]:
sales.loc[sales['OrderDate'].between('2016-07-01', '2016-12-31'),]

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,Year,Month
29519,89,1,2016-07-01,p1030071,1,2450,c127482,2016,7
29520,89,2,2016-07-01,p1144371,1,1250,c127482,2016,7
29521,89,4,2016-07-01,p1159481,1,1650,c127482,2016,7
29522,101,1,2016-07-01,p1011291,1,273,c217179,2016,7
29523,103,1,2016-07-01,p1175481,1,1300,c222561,2016,7
...,...,...,...,...,...,...,...,...,...
55971,97,5,2016-12-31,p1005891,2,3900,c259362,2016,12
55972,99,1,2016-12-31,p1012751,1,1850,c350918,2016,12
55973,99,2,2016-12-31,p1159481,1,2200,c350918,2016,12
55974,99,3,2016-12-31,p1207281,1,3300,c350918,2016,12


In [18]:
sales.loc[sales['OrderDate'].between('2016-07-01', '2016-12-31'), ['CustomerID']].drop_duplicates()

,CustomerID
29519,c127482
29522,c217179
29523,c222561
29526,c333911
29527,c373441
...,...
55591,c153641
55627,c323093
55670,c313636
55836,c188728


In [19]:
customer1 = sales.loc[sales['OrderDate'].between('2016-07-01', '2016-12-31'), ['CustomerID']].drop_duplicates()

In [20]:
customer2 = customers.loc[customers['RegisterDate'].between('2015-01-01', '2016-12-31'), ['CustomerID']]

In [21]:
customer3 = customer1.merge(customer2, how='inner')
customer3.head()

,CustomerID
0,c373441
1,c337831
2,c337999
3,c341674
4,c349783


3-2) 3-1)의 결과에 해당하는 고객ID 별로 2017년 3월 방문 여부(이탈여부) 데이터프레임 만들기
    - 고객ID
    - 이탈여부


In [22]:
customer4 = sales.loc[sales['OrderDate'].between('2017-01-01', '2017-03-31'), ['CustomerID']].drop_duplicates()
customer4.head()

,CustomerID
55976,c271068
55982,c146132
55983,c226242
55984,c140658
55985,c328839


In [23]:
customer4['Churn'] = 0
customer4.head()

,CustomerID,Churn
55976,c271068,0
55982,c146132,0
55983,c226242,0
55984,c140658,0
55985,c328839,0


In [24]:
customer3.merge(customer4, how = 'left')

,CustomerID,Churn
0,c373441,0.0
1,c337831,NaN
2,c337999,0.0
3,c341674,0.0
4,c349783,0.0
...,...,...
349,c396022,0.0
350,c396059,0.0
351,c392672,NaN
352,c345428,NaN


In [25]:
customer_churn = customer3.merge(customer4, how = 'left').fillna(1)

In [26]:
customer_churn.Churn.value_counts()

0.0    226
1.0    128
Name: Churn, dtype: int64

In [28]:
customer_churn

,CustomerID,Churn
0,c373441,0.0
1,c337831,1.0
2,c337999,0.0
3,c341674,0.0
4,c349783,0.0
...,...,...
349,c396022,0.0
350,c396059,0.0
351,c392672,1.0
352,c345428,1.0


### 문제 4. 변수 추가하기

문제 1에서 도출된 요인들을 변수로 만들기  
3-2) 문제 결과에 변수들을 추가하기.

아래 문제들을 풀고, 여러분의 변수들을 추가해 봅시다.  
현재를 2016년12월31일로 간주합니다.

4-1) 성별, 나이 추가하기   
현재를 2016년12월로 간주하여 나이를 계산한다.

4-2) 최근 3개월 구매액, 방문횟수

4-3) 최근 3개월 카테고리별 구매액

In [2]:
products.Category.unique()

NameError: name 'products' is not defined

4-4) 가입 개월수
일수 : 날짜1-날짜2
개월수 : 일수 / np.timedelta64(1,'M'))

5) 그외 여러분이 도출한 요인을 추가 변수로 최소 5개 이상 만드시오.(시간 남으면)

## (추가) 2. 데이터 이해

다시 데이터 이해로 돌아갑니다.  
데이터 준비에서 정리한 데이터셋을 가지고, 가설이 참인지 거짓인지 살펴봅니다.  
이때, 기초통계량과 차트를 이용하게 됩니다. 그리고 이 단계를 특별히 **'탐색적 데이터 분석(EDA)'** 이라고 부릅니다.  
여기서는 몇가지 방법을 가지고 살펴보겠습니다.


In [ ]:
import seaborn as sns

In [ ]:
plt.hist(customer_churn['Age'], color = 'blue', edgecolor = 'black',bins = 20)
plt.show()

In [ ]:
sns.FacetGrid(customer_churn, hue='Churn', size = 5)\
    .map(sns.distplot, 'Age', kde=True, hist=False)
plt.show()

In [ ]:
sns.FacetGrid(customer_churn, hue='Churn', size = 5)\
    .map(sns.distplot, 'Count', kde=True, hist=False)
plt.show()

In [ ]:
gb_churn = customer_churn.groupby("Gender")['Churn'].value_counts().unstack()
gb_churn.plot(kind='bar', stacked='True')
gb_churn.plot(kind='bar')
plt.show()

## 4.모델링

모델링을 위한 전처리
    * Dummry Variable
    * Scaling
   

### 문제 5. Dummy variable만들기


범주형 변수들을 가변수화 합니다.

### Data Split

In [ ]:
# sklearn에서 제공하는 split 함수를 이용해보자. : train_test_split
from sklearn.model_selection import train_test_split

# features와 target 분리
X = customer_churn.drop('Churn', axis=1)
y = customer_churn.loc[:, 'Churn'] 

train_features, test_features, train_target, test_target \
    = train_test_split(X, y, test_size=0.2, random_state=1)

### 문제 6. Scalinng

숫자형 변수들을 스케일링 합니다.

In [ ]:
# 필요한 함수 로딩
from sklearn.preprocessing import MinMaxScaler

# 선언하기
minmax = MinMaxScaler()

# train 데이터로 .fit ==> 모든변수의 min, max, 공식 도출.
# 말하자면, scaling 하는 모델을 만드는 것!
minmax.fit(train_features) # train min, max

# 위 scaling 공식으로 적용하기(변환하기)
train_features_sc = minmax.transform(train_features)
test_features_sc = minmax.transform(test_features)

In [ ]:
train_target, test_target = train_target.values, test_target.values

### 모델링
다음 코드는 그대로 실행하여 결과를 확인해 봅시다.

In [ ]:
# 필요한 라이브러리 가져오자
from sklearn.neighbors import KNeighborsClassifier

# confusion matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# 모델 준비
knn_cl = KNeighborsClassifier(n_neighbors=4)
knn_cl.fit(train_features_sc, train_target)
test_pred = knn_cl.predict(test_features_sc)

test_acc = accuracy_score(test_target, test_pred)
print('test 정확도 : {:.2f}%'.format(test_acc*100))


정 분류율(Accuracy)은 몇 % 인가요?  
높이기 위해서 더 해볼 수 있는 것은?

In [ ]:
print(confusion_matrix(test_target, test_pred))